## Extension to lab 1: Including the prior

In [10]:
import cv2
import scipy.stats as ss
from mpmath import *
import numpy as np
import matplotlib.pyplot as plt
import math
%xmode plain

Exception reporting mode: Plain


In [11]:
def read_img(img_name='train.jpg', mask_name='mask.jpg'):
    
    img = cv2.imread(img_name, 1)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    mask = cv2.imread(mask_name, 1)
    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    
    return np.array(img).astype(float), np.array(mask).astype(float)

In [12]:
img, mask = read_img()

In [13]:
def get_pixels(img, mask, num_pixels):
    threshold = 128
    if num_pixels:
        ncoin = np.sum(mask > threshold )
        ndesk = np.sum(mask <= threshold )
        return ncoin, ndesk
    else:
        coin = img[(mask > threshold).nonzero()]
        desk = img[(mask <= threshold).nonzero()]
        return coin, desk

In [14]:
def ML(img, mask, percentage):
    
    coin_pxl, desk_pxl = get_pixels(img, mask, False)
    coin_pxl = coin_pxl[:,2]
    desk_pxl = desk_pxl[:,2]
    
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))
    
    # sigma calculation functions
    calc_mu =  lambda x: np.mean(x)
    calc_sigma = lambda x, mu: np.mean((x-mu)**2)
    
    # coin paramaters
    coin_mu = calc_mu(coin_pxl[0:num_elements_coin])
    coin_sig = calc_sigma(coin_pxl[0:num_elements_coin], coin_mu)

    # non-coin paramaters
    desk_mu = calc_mu(desk_pxl[0:num_elements_desk])
    desk_sig = calc_sigma(desk_pxl[0:num_elements_desk], desk_mu)
    
    print("{:.2f}% mu coin {:.6f} sigma coin {:.6f} ____ mu coin {:.6f} sigma coin {:.6f}".format(percentage, coin_mu, coin_sig, desk_mu, desk_sig))
    
    return coin_mu, coin_sig, desk_mu, desk_sig

def get_prob(pixels, c_mu, c_var, d_mu, d_var, prior):
    
    p1 = ss.norm.pdf(pixels, c_mu, np.sqrt(c_var))
    p2 = ss.norm.pdf(pixels, d_mu, np.sqrt(d_var))
    
    return (p1*prior)/(p1*(prior) + p2*(1-prior))

def test_model(img, mask, c_mu, c_var, d_mu, d_var, lam6da, eps=1):
    
    img_reshape = img[:,:, 2].flatten()
    mask_reshape = mask.flatten()
    
    chunk_size =int(np.floor(eps*len(img_reshape)))
    pxl = img_reshape[0:chunk_size]
    mask_pxl = mask_reshape[0:chunk_size]
    x = get_prob(pxl, c_mu, c_var, d_mu, d_var, lam6da)
    print(x)
    x[ x > 0.5] = 1
    x[ x <= 0.5] = 0
    mask_pxl[ mask_pxl > 200] = 255
    mask_pxl[ mask_pxl <= 200] = 0
    
    return mask_pxl.reshape((960, 1280))
#     return np.sum(x == mask_pxl)/chunk_size
    

In [15]:
def get_ml_lam6da(train_img, train_mask):
    
    ncoin, ndesk = get_pixels(train_img, train_mask, True)
    print(ncoin / (ncoin + ndesk))
    return ncoin / (ncoin + ndesk)

def get_map_lam6da(train_img, train_mask, alpha, beta):
        
    ncoin, ndesk = get_pixels(train_img, train_mask, True)
    lam6da_h = (ncoin + alpha - 1)/((ncoin + ndesk) + ( alpha + beta) - 2) 
    
    print(lam6da_h)
    
    return lam6da_h

In [ ]:
lam6da = get_ml_lam6da(img, mask)
lam6da_map = get_map_lam6da(img, mask, 1, 1)

0.03838053385416667
0.03838053385416667


In [ ]:
coin_mu, coin_sig, desk_mu, desk_sig = ML(img, mask, 1.0)
out = test_model(img, mask, coin_mu, coin_sig, desk_mu, desk_sig, lam6da, eps=1.0)
cv2.imshow('output', out)
cv2.waitKey(0)
cv2.destroyAllWindows()

1.00% mu coin 202.131568 sigma coin 1234.243387 ____ mu coin 137.204442 sigma coin 337.197810
[0.00273024 0.00295244 0.00308012 ... 0.00322021 0.00295244 0.00273024]


In [ ]:
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
print("v image 1 ML: ", test_model(img1, mask1, coin_mu, coin_sig, desk_mu, desk_sig, lam6da, eps=1))
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
print("v image 1 MAP: ", test_model(img1, mask1, coin_mu, coin_sig, desk_mu, desk_sig, lam6da_map, eps=1))
###################################
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
print("v image 2 ML: ", test_model(img2, mask2, coin_mu, coin_sig, desk_mu, desk_sig, lam6da, eps=1))
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
print("v image 2 MAP: ", test_model(img2, mask2, coin_mu, coin_sig, desk_mu, desk_sig, lam6da, eps=1))

In [ ]:
def MAP(img, mask, percentage):
    
    coin_pxl, desk_pxl = get_pixels(img, mask, False)
    coin_pxl = coin_pxl[:,2]
    desk_pxl = desk_pxl[:,2]
    
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))    
    
    #sigma calculation functions
    calc_mu = lambda x: (np.sum(x))/(len(x)+1)
    calc_sigma = lambda x, mu: (np.sum((x-mu)**2) + 2 + (-mu)**2)/(len(x) + 5)
    
    # coin paramaters
    coin_mu = calc_mu(coin_pxl[0:num_elements_coin])
    coin_sig = calc_sigma(coin_pxl[0:num_elements_coin], coin_mu)

    # non-coin paramaters
    desk_mu = calc_mu(desk_pxl[0:num_elements_desk])
    desk_sig = calc_sigma(desk_pxl[0:num_elements_desk], desk_mu)
    
    print("{:.2f}% mu coin {:.6f} sigma coin {:.6f} ____ mu coin {:.6f} sigma coin {:.6f}".format(percentage, coin_mu, coin_sig, desk_mu, desk_sig))
    
    return coin_mu, coin_sig, desk_mu, desk_sig

In [ ]:
coin_mu1, coin_sig1, desk_mu1, desk_sig1 = MAP(img, mask, 1)
out = test_model(img, mask, coin_mu1, coin_sig1, desk_mu1, desk_sig1, lam6da,eps=1)
cv2.imshow('output', out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
print("validation image 1: ", test_model(img1, mask1, coin_mu1, coin_sig1, desk_mu1, desk_sig1, eps=0.5))
###################################
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
print("validation image 2: ", test_model(img2, img2, coin_mu1, coin_sig1, desk_mu1, desk_sig1, eps=0.5))